# BindingDB Dataset Exploration for Protein-Ligand Binding Affinity Prediction

This notebook explores the BindingDB dataset to prepare data for pose-free protein-ligand binding affinity prediction. We'll focus on combining protein structure information with SMILES molecular representations for a lightweight AI approach to drug discovery.

## Project Goals:
- **Target**: Ligand-Protein Binding Affinity Estimation (Mini AlphaFold Challenge)
- **Approach**: Pose-free prediction using protein sequences/structures + SMILES codes
- **Dataset**: BindingDB - binding affinities for protein-ligand pairs
- **Model Constraint**: ≤50M parameters for hackathon feasibility

## 1. Install and Import Required Libraries

We'll install and import the necessary libraries for data processing, molecular handling, and visualization.

In [ ]:
# Install required packages
# Uncomment and run these lines if packages are not installed
# !pip install pandas numpy matplotlib seaborn
# !pip install rdkit-pypi  # For SMILES processing
# !pip install biopython  # For protein sequence handling
# !pip install requests   # For downloading data
# !pip install scikit-learn  # For ML utilities

In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import gzip
import io
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import molecular handling libraries
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors, rdMolDescriptors
    from rdkit.Chem import AllChem
    print("✓ RDKit imported successfully")
except ImportError:
    print("❌ RDKit not available. Install with: pip install rdkit-pypi")

# Import bioinformatics libraries
try:
    from Bio.SeqUtils.ProtParam import ProteinAnalysis
    from Bio.SeqUtils import molecular_weight
    print("✓ BioPython imported successfully")
except ImportError:
    print("❌ BioPython not available. Install with: pip install biopython")

# Import ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

print("✓ Core libraries imported successfully")

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

## 2. Download and Load BindingDB Dataset

BindingDB is a comprehensive database of measured binding affinities for protein-ligand pairs. We'll download a subset of the data focusing on entries with both protein structure information and SMILES codes.

In [ ]:
# Create data directory
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# BindingDB download URL (TSV format with filtered data)
# This URL provides a subset of BindingDB with Ki, Kd, and IC50 values
bindingdb_url = "https://www.bindingdb.org/bind/downloads/BindingDB_All_202310_tsv.zip"

# For demo purposes, we'll use a smaller subset or create sample data
# In practice, you'd download the full dataset

def download_bindingdb_sample():
    """
    Download a sample of BindingDB data for exploration
    Note: The full dataset is very large (~2GB), so we'll start with a subset
    """
    print("Creating sample BindingDB data for exploration...")
    
    # Sample data structure based on BindingDB format
    sample_data = {
        'Target Name': ['Human albumin', 'HIV-1 protease', 'Thrombin', 'Acetylcholinesterase', 'Carbonic anhydrase II'] * 200,
        'UniProt (SwissProt) Primary ID of Target Chain': ['P02768', 'P03366', 'P00734', 'P22303', 'P00918'] * 200,
        'PDB ID(s) for Ligand-Target Complex': ['4F5S', '1HPV', '1PPE', '1ACJ', '2CBA'] * 200,
        'Ligand SMILES': [
            'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O',  # Ibuprofen-like
            'CC(C)(C)NC(=O)C1=CC=CC=C1',        # Simple amide
            'CCN(CC)CCOC(=O)C1=CC=CC=C1',      # Ester compound
            'CC1=CC=C(C=C1)S(=O)(=O)N',        # Sulfonamide
            'CC(=O)NC1=CC=C(C=C1)O'            # Acetaminophen-like
        ] * 200,
        'Ki (nM)': np.random.lognormal(5, 2, 1000),  # Log-normal distribution typical for Ki values
        'Kd (nM)': np.random.lognormal(4.5, 1.8, 1000),
        'IC50 (nM)': np.random.lognormal(5.5, 2.2, 1000),
        'Target Source Organism According to Curator or DataSource': ['Homo sapiens'] * 1000,
        'Ligand INCHI Key': ['INCHIKEY-' + str(i).zfill(6) for i in range(1000)],
        'Number of Protein Chains in Target (>1 implies a multichain complex)': [1] * 800 + [2] * 150 + [4] * 50,
        'Molecular Weight of Ligand (g/mol)': np.random.normal(350, 150, 1000).clip(50, 800)
    }
    
    df = pd.DataFrame(sample_data)
    
    # Add some missing values to simulate real data
    df.loc[np.random.choice(df.index, 100, replace=False), 'Ki (nM)'] = np.nan
    df.loc[np.random.choice(df.index, 120, replace=False), 'Kd (nM)'] = np.nan
    df.loc[np.random.choice(df.index, 80, replace=False), 'IC50 (nM)'] = np.nan
    df.loc[np.random.choice(df.index, 50, replace=False), 'PDB ID(s) for Ligand-Target Complex'] = ''
    
    return df

# Load the sample dataset
print("Loading BindingDB sample data...")
df_bindingdb = download_bindingdb_sample()

print(f"✓ Loaded sample dataset with {len(df_bindingdb)} entries")
print(f"Dataset shape: {df_bindingdb.shape}")
print("\nColumn names:")
for i, col in enumerate(df_bindingdb.columns, 1):
    print(f"{i:2d}. {col}")

## 3. Data Exploration and Statistics

Let's explore the dataset structure, distribution of binding affinities, and identify data quality patterns that will be important for our pose-free binding affinity prediction model.

In [ ]:
# Basic dataset information
print("=== BindingDB Dataset Overview ===")
print(f"Total entries: {len(df_bindingdb):,}")
print(f"Total columns: {len(df_bindingdb.columns)}")
print()

# Display first few rows
print("=== Sample Data ===")
display(df_bindingdb.head())
print()

# Data types and missing values
print("=== Data Quality Summary ===")
info_df = pd.DataFrame({
    'Column': df_bindingdb.columns,
    'Data Type': df_bindingdb.dtypes,
    'Non-Null Count': df_bindingdb.count(),
    'Missing Count': df_bindingdb.isnull().sum(),
    'Missing %': (df_bindingdb.isnull().sum() / len(df_bindingdb) * 100).round(2)
})
display(info_df)

# Unique proteins and ligands
print("\n=== Dataset Diversity ===")
print(f"Unique target proteins: {df_bindingdb['Target Name'].nunique()}")
print(f"Unique UniProt IDs: {df_bindingdb['UniProt (SwissProt) Primary ID of Target Chain'].nunique()}")
print(f"Unique PDB structures: {df_bindingdb[df_bindingdb['PDB ID(s) for Ligand-Target Complex'] != '']['PDB ID(s) for Ligand-Target Complex'].nunique()}")
print(f"Unique SMILES: {df_bindingdb['Ligand SMILES'].nunique()}")

# Target protein distribution
print("\n=== Top 10 Most Studied Proteins ===")
target_counts = df_bindingdb['Target Name'].value_counts().head(10)
print(target_counts)

In [ ]:
# Visualize binding affinity distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Binding Affinity Distributions in BindingDB Sample', fontsize=16)

# Ki distribution
ki_data = df_bindingdb['Ki (nM)'].dropna()
axes[0, 0].hist(np.log10(ki_data), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('log10(Ki) [nM]')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title(f'Ki Distribution (n={len(ki_data)})')
axes[0, 0].grid(True, alpha=0.3)

# Kd distribution
kd_data = df_bindingdb['Kd (nM)'].dropna()
axes[0, 1].hist(np.log10(kd_data), bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0, 1].set_xlabel('log10(Kd) [nM]')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title(f'Kd Distribution (n={len(kd_data)})')
axes[0, 1].grid(True, alpha=0.3)

# IC50 distribution
ic50_data = df_bindingdb['IC50 (nM)'].dropna()
axes[1, 0].hist(np.log10(ic50_data), bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 0].set_xlabel('log10(IC50) [nM]')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title(f'IC50 Distribution (n={len(ic50_data)})')
axes[1, 0].grid(True, alpha=0.3)

# Molecular weight distribution
mw_data = df_bindingdb['Molecular Weight of Ligand (g/mol)'].dropna()
axes[1, 1].hist(mw_data, bins=30, alpha=0.7, color='gold', edgecolor='black')
axes[1, 1].set_xlabel('Molecular Weight [g/mol]')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title(f'Ligand MW Distribution (n={len(mw_data)})')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print("=== Binding Affinity Summary Statistics ===")
affinity_cols = ['Ki (nM)', 'Kd (nM)', 'IC50 (nM)']
summary_stats = df_bindingdb[affinity_cols].describe()
print(summary_stats)

# Data availability for each affinity type
print("\n=== Data Availability ===")
for col in affinity_cols:
    available = df_bindingdb[col].notna().sum()
    total = len(df_bindingdb)
    print(f"{col}: {available:,}/{total:,} ({available/total*100:.1f}%)")

## 4. Filter Data for Binding Affinity Tasks

Now we'll filter the dataset to create a high-quality subset suitable for machine learning. We'll focus on entries with valid SMILES codes, protein structure information, and reliable binding affinity measurements.

In [ ]:
# Define filtering criteria for high-quality data
def filter_bindingdb_data(df):
    """
    Filter BindingDB data for machine learning tasks
    """
    print("=== Data Filtering Pipeline ===")
    print(f"Starting with {len(df):,} entries")
    
    # Step 1: Remove entries without SMILES
    df_filtered = df[df['Ligand SMILES'].notna() & (df['Ligand SMILES'] != '')].copy()
    print(f"After removing entries without SMILES: {len(df_filtered):,} entries")
    
    # Step 2: Remove entries without UniProt ID (protein identifier)
    df_filtered = df_filtered[df_filtered['UniProt (SwissProt) Primary ID of Target Chain'].notna()].copy()
    print(f"After removing entries without UniProt ID: {len(df_filtered):,} entries")
    
    # Step 3: Keep entries with at least one binding affinity measurement
    has_affinity = (df_filtered['Ki (nM)'].notna() | 
                   df_filtered['Kd (nM)'].notna() | 
                   df_filtered['IC50 (nM)'].notna())
    df_filtered = df_filtered[has_affinity].copy()
    print(f"After requiring at least one affinity measurement: {len(df_filtered):,} entries")
    
    # Step 4: Filter by molecular weight (typical drug-like range)
    mw_filter = (df_filtered['Molecular Weight of Ligand (g/mol)'] >= 100) & \
                (df_filtered['Molecular Weight of Ligand (g/mol)'] <= 1000)
    df_filtered = df_filtered[mw_filter].copy()
    print(f"After molecular weight filter (100-1000 g/mol): {len(df_filtered):,} entries")
    
    # Step 5: Remove extreme outliers in binding affinity
    for col in ['Ki (nM)', 'Kd (nM)', 'IC50 (nM)']:
        if col in df_filtered.columns:
            # Remove values outside reasonable range (0.1 nM to 100 μM = 100,000 nM)
            valid_range = (df_filtered[col] >= 0.1) & (df_filtered[col] <= 100000)
            df_filtered.loc[~valid_range, col] = np.nan
    
    # Step 6: Focus on human proteins for this demo
    df_filtered = df_filtered[df_filtered['Target Source Organism According to Curator or DataSource'] == 'Homo sapiens'].copy()
    print(f"After filtering for human proteins: {len(df_filtered):,} entries")
    
    return df_filtered

# Apply filtering
df_clean = filter_bindingdb_data(df_bindingdb)

print(f"\n=== Final Dataset Summary ===")
print(f"Clean dataset size: {len(df_clean):,} entries")
print(f"Unique proteins: {df_clean['Target Name'].nunique()}")
print(f"Unique ligands: {df_clean['Ligand SMILES'].nunique()}")

# Check data availability after filtering
print("\n=== Affinity Data Availability (Clean Dataset) ===")
for col in ['Ki (nM)', 'Kd (nM)', 'IC50 (nM)']:
    available = df_clean[col].notna().sum()
    total = len(df_clean)
    print(f"{col}: {available:,}/{total:,} ({available/total*100:.1f}%)")

## 5. Protein Structure Data Processing

For pose-free binding affinity prediction, we need to extract meaningful features from protein sequences and structures. We'll create protein representations that can be used without requiring 3D coordinates.

In [ ]:
# Create sample protein sequences for demonstration
# In practice, you would fetch these from UniProt or PDB
def create_sample_protein_features(df):
    """
    Create sample protein features for demonstration
    In a real implementation, you would:
    1. Fetch protein sequences from UniProt using the UniProt IDs
    2. Extract features from the sequences using BioPython
    3. Optionally use pre-trained protein embeddings (ESM-2, ProtBERT)
    """
    
    # Sample protein sequences (shortened for demo)
    protein_sequences = {
        'P02768': 'MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCVADESAENCDKSLHTLFGDKLCTVATLRETYGEMADCCAKQEPERNECFLQHKDDNPNLPRLVRPEVDVMCTAFHDNEETFLKKYLYEIARRHPYFYAPELLFFAKRYKAAFTECCQAADKAACLLPKLDELRDEGKASSAKQRLKCASLQKFGERAFKAWAVARLSQRFPKAEFAEVSKLVTDLTKVHTECCHGDLLECADDRADLAKYICENQDSISSKLKECCEKPLLEKSHCIAEVENDEMPADLPSLAADFVESKDVCKNYAEAKDVFLGMFLYEYARRHPDYSVVLLLRLAKTYETTLEKCCAAADPHECYAKVFDEFKPLVEEPQNLIKQNCELFEQLGEYKFQNALLVRYTKKVPQVSTPTLVEVSRNLGKVGSKCCKHPEAKRMPCAEDYLSVVLNQLCVLHEKTPVSDRVTKCCTESLVNRRPCFSALEVDETYVPKEFNAETFTFHADICTLSEKERQIKKQTALVELVKHKPKATKEQLKAVMDDFAAFVEKCCKADDKETCFAEEGKKLVAASQAALGL',
        'P03366': 'PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKMIGGIGGFIKVRQYDQILIEICGHKAIGTVLVGPTPVNIIGRNLLTQIGCTLNFPISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGKISKIGPENPYNTPVFAIKKKDSTKWRKLVDFRELNKRTQDFWEVQLGIPHPAGLKKKKSVTVLDVGDAYFSVPLDEDFRKYTAFTIPSINNETPGIRYQYNVLPQGWKGSPAIFQSSMTKILEPFRKQNPDIVIYQYMDDLYVGSDLEIGQHRTKIEELRQHLLRWGLTTPDKKHQKEPPFLWMGYELHPDKWTVQPIVLPEKDSWTVNDIQKLVGKLNWASQIYPGIKVRQLCKLLRGTKALTEVIPLTEEAELELAENREILKEPVHGVYYDPSKDLIAEIQKQGQGQWTYQIYQEPFKNLKTGKYARMRGAHTNDVKQLTEAVQKITTESIVIWGKTPKFKLPIQKETWETWWTEYWQATWIPEWEFVNTPPLVKLWYQLEKEPIVGAETFYVDGAANRETKLGKAGYVTNRGRQKVVTLTDTTNQKTELQAIYLALQDSGLEVNIVTDSQYALGIIQAQPDQSESELVNQIIEQLINKEKVYLAWVPAHKGIGGNEQVDKLVSAGIRKVLFLDGIDKAQEEHEKYHSNWRAMASDFNLPPVVAKEIVASCDKCQLKGEAMHGQVDCSPGIWQLDCTHLEGKVILVAVHVASGYIEAEVIPAETGQETAYFLLKLAGRWPVKTIHTDNGSNFTGATVRAACWWAGIKQEFGIPYNPQSQGVVESMNKELKKIIGQVRDQAEHLKTAVQMAVFIHNFKRKGGIGGYSAGERIVDIIATDIQTKELQKQITKIQNFRVYYRDSRDPLWKGPAKLLWKGEGAVVIQDNSDIKVVPRRKAKIIRDYGKQMAGDDCVASRQDED',
        'P00734': 'MNKPLLLVAILLVLASLCHATFWQSLRQSHPDSTDHMKPLPWPKTLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKMIGGIGGFIKVRQYDQILIEICGHKAIGTVLVGPTPVNIIGRNLLTQIGCTLNFPISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGKISKIGPENPYNTPVFAIKKKDSTKWRKLVDFRELNKRTQDFWEVQLGIPHPAGLKKKKSVTVLDVGDAYFSVPLDEDFRKYTAFTIPSINNETPGIRYQYNVLPQGWKGSPAIFQSSMTKILEPFRKQNPDIVIYQYMDDLYVGSDLEIGQHRTKIEELRQHLLRWGLTTPDKKHQKEPPFLWMGYELHPDKWTVQPIVLPEKDSWTVNDIQKLVGKLNWASQIYPGIKVRQLCKLLRGTKALTEVIPLTEEAELELAENREILKEPVHGVYYDPSKDLIAEIQKQGQGQWTYQIYQEPFKNLKTGKYARMRGAHTNDVKQLTEAVQKITTESIVIWGKTPKFKLPIQKETWETWWTEYWQATWIPEWEFVNTPPLVKLWYQLEKEPIVGAETFYVDGAANRETKLGKAGYVTNRGRQKVVTLTDTTNQKTELQAIYLALQDSGLEVNIVTDSQYALGIIQAQPDQSESELVNQIIEQLINKEKVYLAWVPAHKGIGGNEQVDKLVSAGIRKVLFLDGIDKAQEEHEKYHSNWRAMASDFNLPPVVAKEIVASCDKCQLKGEAMHGQVDCSPGIWQLDCTHLEGKVILVAVHVASGYIEAEVIPAETGQETAYFLLKLAGRWPVKTIHTDNGSNFTGATVRAACWWAGIKQEFGIPYNPQSQGVVESMNKELKKIIGQVRDQAEHLKTAVQMAVFIHNFKRKGGIGGYSAGERIVDIIATDIQTKELQKQITKIQNFRVYYRDSRDPLWKGPAKLLWKGEGAVVIQDNSDIKVVPRRKAKIIRDYGKQMAGDDCVASRQDED',
        'P22303': 'MELFRPHFLLIRCLPLALLCLLPSLALAEDRSLLPKLHYFNARGRQVGLNLTGGGSVGAPLVLAVALLIAAALLPSEQARAAGRRLRRLLLAALLLGAGGGGLLLLLLLLLAALLLLLLGGGGGGLLLLAALLLLAAALLLLAAGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLLGGGGGGLLLLAALLLLAAAALLL',
        'P00918': 'MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFNGEGEPEELMVDNWRPAQPLKNRQIKASFK'
    }
    
    protein_features = []
    
    for idx, row in df.iterrows():
        uniprot_id = row['UniProt (SwissProt) Primary ID of Target Chain']
        
        # Get sequence (in practice, fetch from UniProt)
        sequence = protein_sequences.get(uniprot_id, protein_sequences['P02768'])  # Default to albumin
        
        if len(sequence) > 500:  # Truncate very long sequences for demo
            sequence = sequence[:500]
        
        try:
            # Calculate basic protein features using BioPython
            analysis = ProteinAnalysis(sequence)
            
            features = {
                'uniprot_id': uniprot_id,
                'sequence_length': len(sequence),
                'molecular_weight': analysis.molecular_weight(),
                'aromaticity': analysis.aromaticity(),
                'instability_index': analysis.instability_index(),
                'isoelectric_point': analysis.isoelectric_point(),
                'gravy': analysis.gravy(),  # Grand average of hydropathy
            }
            
            # Add amino acid composition
            aa_percent = analysis.get_amino_acids_percent()
            for aa, percent in aa_percent.items():
                features[f'aa_{aa}_percent'] = percent
            
            # Add secondary structure fraction
            sec_struct = analysis.secondary_structure_fraction()
            features['helix_fraction'] = sec_struct[0]
            features['turn_fraction'] = sec_struct[1] 
            features['sheet_fraction'] = sec_struct[2]
            
            protein_features.append(features)
            
        except Exception as e:
            print(f"Error processing protein {uniprot_id}: {e}")
            # Add default features for failed cases
            protein_features.append({
                'uniprot_id': uniprot_id,
                'sequence_length': 300,
                'molecular_weight': 35000,
                'aromaticity': 0.1,
                'instability_index': 40,
                'isoelectric_point': 7.0,
                'gravy': -0.5,
                'helix_fraction': 0.3,
                'turn_fraction': 0.3,
                'sheet_fraction': 0.4
            })
    
    return pd.DataFrame(protein_features)

# Extract protein features
print("Extracting protein features...")
protein_features_df = create_sample_protein_features(df_clean.head(100))  # Process first 100 for demo

print(f"✓ Extracted features for {len(protein_features_df)} proteins")
print("\nProtein features shape:", protein_features_df.shape)
print("\nSample protein features:")
display(protein_features_df.head())

# Visualize protein feature distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Protein Feature Distributions', fontsize=16)

# Key protein features to visualize
features_to_plot = [
    ('sequence_length', 'Sequence Length'),
    ('molecular_weight', 'Molecular Weight (Da)'),
    ('isoelectric_point', 'Isoelectric Point'),
    ('aromaticity', 'Aromaticity'),
    ('gravy', 'GRAVY (Hydropathy)'),
    ('instability_index', 'Instability Index')
]

for i, (feature, title) in enumerate(features_to_plot):
    row, col = i // 3, i % 3
    axes[row, col].hist(protein_features_df[feature], bins=20, alpha=0.7, edgecolor='black')
    axes[row, col].set_title(title)
    axes[row, col].set_xlabel(feature.replace('_', ' ').title())
    axes[row, col].set_ylabel('Frequency')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. SMILES Molecular Representation Processing

SMILES (Simplified Molecular Input Line Entry System) codes represent the chemical structure of molecules as strings. We'll process these to extract molecular features and fingerprints for our pose-free binding prediction model.

In [ ]:
# Process SMILES strings and extract molecular features
def process_smiles_features(df, max_molecules=100):
    """
    Process SMILES strings and extract molecular descriptors using RDKit
    """
    print("Processing SMILES strings...")
    
    molecular_features = []
    valid_smiles = []
    
    for idx, smiles in enumerate(df['Ligand SMILES'].head(max_molecules)):
        if pd.isna(smiles) or smiles == '':
            continue
            
        try:
            # Parse SMILES
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                continue
                
            # Calculate molecular descriptors
            features = {
                'smiles': smiles,
                'mol_weight': Descriptors.MolWt(mol),
                'logp': Descriptors.MolLogP(mol),
                'num_h_donors': Descriptors.NumHDonors(mol),
                'num_h_acceptors': Descriptors.NumHAcceptors(mol),
                'num_rotatable_bonds': Descriptors.NumRotatableBonds(mol),
                'tpsa': Descriptors.TPSA(mol),  # Topological polar surface area
                'num_aromatic_rings': Descriptors.NumAromaticRings(mol),
                'num_aliphatic_rings': Descriptors.NumAliphaticRings(mol),
                'num_heteroatoms': Descriptors.NumHeteroatoms(mol),
                'num_heavy_atoms': Descriptors.HeavyAtomCount(mol),
                'fraction_csp3': Descriptors.FractionCsp3(mol),
                'bertz_ct': Descriptors.BertzCT(mol),  # Complexity
            }
            
            # Calculate molecular fingerprints (Morgan/ECFP)
            fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
            fp_array = np.array(fingerprint)
            
            # Add fingerprint features
            for i in range(min(50, len(fp_array))):  # Use first 50 bits for demo
                features[f'fp_bit_{i}'] = fp_array[i]
            
            molecular_features.append(features)
            valid_smiles.append(smiles)
            
        except Exception as e:
            print(f"Error processing SMILES {smiles}: {e}")
            continue
    
    print(f"✓ Successfully processed {len(molecular_features)} molecules")
    return pd.DataFrame(molecular_features)

# Process molecular features
if 'Chem' in globals():
    mol_features_df = process_smiles_features(df_clean)
    
    print(f"Molecular features shape: {mol_features_df.shape}")
    print("\nSample molecular features:")
    display(mol_features_df[['smiles', 'mol_weight', 'logp', 'num_h_donors', 'num_h_acceptors', 'tpsa']].head())
    
    # Visualize molecular property distributions
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Molecular Property Distributions', fontsize=16)
    
    # Key molecular properties
    properties_to_plot = [
        ('mol_weight', 'Molecular Weight (Da)'),
        ('logp', 'LogP (Lipophilicity)'),
        ('num_h_donors', 'H-Bond Donors'),
        ('num_h_acceptors', 'H-Bond Acceptors'),
        ('tpsa', 'TPSA (Ų)'),
        ('num_rotatable_bonds', 'Rotatable Bonds')
    ]
    
    for i, (prop, title) in enumerate(properties_to_plot):
        row, col = i // 3, i % 3
        axes[row, col].hist(mol_features_df[prop], bins=20, alpha=0.7, edgecolor='black')
        axes[row, col].set_title(title)
        axes[row, col].set_xlabel(prop.replace('_', ' ').title())
        axes[row, col].set_ylabel('Frequency')
        axes[row, col].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Lipinski's Rule of Five analysis
    print("\n=== Lipinski's Rule of Five Analysis ===")
    lipinski_violations = 0
    total_molecules = len(mol_features_df)
    
    rule_violations = {
        'MW > 500': (mol_features_df['mol_weight'] > 500).sum(),
        'LogP > 5': (mol_features_df['logp'] > 5).sum(),
        'H-donors > 5': (mol_features_df['num_h_donors'] > 5).sum(),
        'H-acceptors > 10': (mol_features_df['num_h_acceptors'] > 10).sum()
    }
    
    for rule, violations in rule_violations.items():
        percentage = violations / total_molecules * 100
        print(f"{rule}: {violations}/{total_molecules} ({percentage:.1f}%)")
    
    # Count molecules with 0, 1, 2+ violations
    violations_per_mol = (
        (mol_features_df['mol_weight'] > 500) +
        (mol_features_df['logp'] > 5) +
        (mol_features_df['num_h_donors'] > 5) +
        (mol_features_df['num_h_acceptors'] > 10)
    ).astype(int)
    
    print(f"\nMolecules with 0 violations (drug-like): {(violations_per_mol == 0).sum()}/{total_molecules} ({(violations_per_mol == 0).sum()/total_molecules*100:.1f}%)")
    print(f"Molecules with 1 violation: {(violations_per_mol == 1).sum()}/{total_molecules} ({(violations_per_mol == 1).sum()/total_molecules*100:.1f}%)")
    print(f"Molecules with 2+ violations: {(violations_per_mol >= 2).sum()}/{total_molecules} ({(violations_per_mol >= 2).sum()/total_molecules*100:.1f}%)")
    
else:
    print("❌ RDKit not available - skipping molecular feature extraction")
    print("Install RDKit with: pip install rdkit-pypi")

## 7. Prepare Training/Validation Splits

For reliable model evaluation, we need to create train/validation/test splits that avoid data leakage. We'll ensure that similar proteins or molecules don't appear across different splits.

In [ ]:
# Prepare training/validation splits with no data leakage
def create_train_val_splits(df, test_size=0.2, val_size=0.2, random_state=42):
    """
    Create train/val/test splits avoiding data leakage
    Split by protein families to ensure no similar proteins across splits
    """
    print("=== Creating Train/Validation/Test Splits ===")
    
    # For demonstration, we'll split by target protein
    # In practice, you might use protein family clustering
    unique_proteins = df['UniProt (SwissProt) Primary ID of Target Chain'].unique()
    
    # Split proteins into train/val/test
    proteins_train, proteins_temp = train_test_split(
        unique_proteins, test_size=(test_size + val_size), random_state=random_state
    )
    
    proteins_val, proteins_test = train_test_split(
        proteins_temp, test_size=(test_size / (test_size + val_size)), random_state=random_state
    )
    
    # Assign data points to splits based on protein
    train_mask = df['UniProt (SwissProt) Primary ID of Target Chain'].isin(proteins_train)
    val_mask = df['UniProt (SwissProt) Primary ID of Target Chain'].isin(proteins_val)
    test_mask = df['UniProt (SwissProt) Primary ID of Target Chain'].isin(proteins_test)
    
    df_train = df[train_mask].copy()
    df_val = df[val_mask].copy()
    df_test = df[test_mask].copy()
    
    print(f"Protein splits:")
    print(f"  Train proteins: {len(proteins_train)} ({len(df_train)} data points)")
    print(f"  Val proteins: {len(proteins_val)} ({len(df_val)} data points)")
    print(f"  Test proteins: {len(proteins_test)} ({len(df_test)} data points)")
    
    return df_train, df_val, df_test

# Create splits
df_train, df_val, df_test = create_train_val_splits(df_clean)

# Create target variable (binding affinity)
def create_target_variable(df):
    """
    Create a unified binding affinity target variable
    Priority: Ki > Kd > IC50
    Convert to pKi/pKd/pIC50 (negative log10) for better model performance
    """
    target = pd.Series(index=df.index, dtype=float)
    
    # Use Ki first (most direct measure)
    ki_mask = df['Ki (nM)'].notna()
    target[ki_mask] = -np.log10(df.loc[ki_mask, 'Ki (nM)'] * 1e-9)  # Convert to pKi
    
    # Use Kd if Ki not available
    kd_mask = target.isna() & df['Kd (nM)'].notna()
    target[kd_mask] = -np.log10(df.loc[kd_mask, 'Kd (nM)'] * 1e-9)  # Convert to pKd
    
    # Use IC50 if neither Ki nor Kd available
    ic50_mask = target.isna() & df['IC50 (nM)'].notna()
    target[ic50_mask] = -np.log10(df.loc[ic50_mask, 'IC50 (nM)'] * 1e-9)  # Convert to pIC50
    
    return target

# Create target variables for each split
y_train = create_target_variable(df_train)
y_val = create_target_variable(df_val)
y_test = create_target_variable(df_test)

# Remove samples without target values
train_valid_mask = y_train.notna()
val_valid_mask = y_val.notna()
test_valid_mask = y_test.notna()

df_train_final = df_train[train_valid_mask].copy()
df_val_final = df_val[val_valid_mask].copy()
df_test_final = df_test[test_valid_mask].copy()

y_train_final = y_train[train_valid_mask]
y_val_final = y_val[val_valid_mask]
y_test_final = y_test[test_valid_mask]

print(f"\n=== Final Dataset Sizes ===")
print(f"Train: {len(df_train_final)} samples")
print(f"Validation: {len(df_val_final)} samples")
print(f"Test: {len(df_test_final)} samples")

# Visualize target variable distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Binding Affinity Target Variable Distribution (pKi/pKd/pIC50)', fontsize=14)

for i, (split_name, y_data) in enumerate([('Train', y_train_final), ('Validation', y_val_final), ('Test', y_test_final)]):
    axes[i].hist(y_data, bins=20, alpha=0.7, edgecolor='black')
    axes[i].set_title(f'{split_name} (n={len(y_data)})')
    axes[i].set_xlabel('pKi/pKd/pIC50')
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)
    
    # Add statistics
    mean_val = y_data.mean()
    std_val = y_data.std()
    axes[i].axvline(mean_val, color='red', linestyle='--', alpha=0.8, label=f'Mean: {mean_val:.2f}')
    axes[i].legend()

plt.tight_layout()
plt.show()

print(f"\n=== Target Variable Statistics ===")
for split_name, y_data in [('Train', y_train_final), ('Validation', y_val_final), ('Test', y_test_final)]:
    print(f"{split_name}: mean={y_data.mean():.2f}, std={y_data.std():.2f}, min={y_data.min():.2f}, max={y_data.max():.2f}")

## 8. Feature Engineering for Pose-Free Prediction

Now we'll combine protein and ligand features into input vectors suitable for machine learning models. This pose-free approach avoids the need for 3D structural information and focuses on sequence-based and chemical descriptors.

In [ ]:
# Combine protein and molecular features for pose-free prediction
def create_combined_features(df_split, protein_features_df, mol_features_df):
    """
    Combine protein and molecular features for each protein-ligand pair
    """
    combined_features = []
    
    for idx, row in df_split.iterrows():
        uniprot_id = row['UniProt (SwissProt) Primary ID of Target Chain']
        smiles = row['Ligand SMILES']
        
        # Get protein features
        protein_row = protein_features_df[protein_features_df['uniprot_id'] == uniprot_id]
        if len(protein_row) == 0:
            # Use default protein features if not found
            protein_feats = {
                'sequence_length': 300, 'molecular_weight': 35000, 'aromaticity': 0.1,
                'instability_index': 40, 'isoelectric_point': 7.0, 'gravy': -0.5,
                'helix_fraction': 0.3, 'turn_fraction': 0.3, 'sheet_fraction': 0.4
            }
        else:
            protein_feats = protein_row.iloc[0].to_dict()
            del protein_feats['uniprot_id']  # Remove ID column
        
        # Get molecular features
        mol_row = mol_features_df[mol_features_df['smiles'] == smiles]
        if len(mol_row) == 0:
            # Use default molecular features if not found
            mol_feats = {
                'mol_weight': 300, 'logp': 2.0, 'num_h_donors': 2, 'num_h_acceptors': 4,
                'num_rotatable_bonds': 3, 'tpsa': 60, 'num_aromatic_rings': 1,
                'num_aliphatic_rings': 0, 'num_heteroatoms': 3, 'num_heavy_atoms': 20
            }
        else:
            mol_feats = mol_row.iloc[0].to_dict()
            del mol_feats['smiles']  # Remove SMILES column
        
        # Combine features
        combined_feats = {**protein_feats, **mol_feats}
        
        # Add interaction features (simple examples)
        combined_feats['protein_mol_mw_ratio'] = protein_feats.get('molecular_weight', 35000) / mol_feats.get('mol_weight', 300)
        combined_feats['hydrophobic_match'] = protein_feats.get('gravy', -0.5) * mol_feats.get('logp', 2.0)
        
        combined_features.append(combined_feats)
    
    return pd.DataFrame(combined_features)

# Create combined feature matrices for each split
print("Creating combined feature matrices...")

if 'protein_features_df' in globals() and 'mol_features_df' in globals():
    X_train = create_combined_features(df_train_final, protein_features_df, mol_features_df)
    X_val = create_combined_features(df_val_final, protein_features_df, mol_features_df)
    X_test = create_combined_features(df_test_final, protein_features_df, mol_features_df)
    
    print(f"✓ Created feature matrices:")
    print(f"  Train: {X_train.shape}")
    print(f"  Validation: {X_val.shape}")
    print(f"  Test: {X_test.shape}")
    
    # Display feature summary
    print(f"\nTotal features: {X_train.shape[1]}")
    print("\nSample features:")
    feature_sample = X_train.head()
    display(feature_sample.iloc[:, :10])  # Show first 10 features
    
    # Scale features for machine learning
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.fillna(0))
    X_val_scaled = scaler.transform(X_val.fillna(0))
    X_test_scaled = scaler.transform(X_test.fillna(0))
    
    print(f"\n✓ Features scaled and ready for machine learning")
    print(f"Feature matrix shapes after scaling:")
    print(f"  X_train: {X_train_scaled.shape}")
    print(f"  X_val: {X_val_scaled.shape}") 
    print(f"  X_test: {X_test_scaled.shape}")
    
    # Feature importance analysis (correlation with target)
    feature_correlations = []
    for i, feature_name in enumerate(X_train.columns):
        corr = np.corrcoef(X_train_scaled[:, i], y_train_final)[0, 1]
        if not np.isnan(corr):
            feature_correlations.append((feature_name, abs(corr)))
    
    # Sort by absolute correlation
    feature_correlations.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\n=== Top 10 Features by Correlation with Binding Affinity ===")
    for i, (feature, corr) in enumerate(feature_correlations[:10]):
        print(f"{i+1:2d}. {feature}: {corr:.3f}")
    
else:
    print("❌ Protein or molecular features not available")
    print("Create sample feature matrices for demonstration...")
    
    # Create minimal feature matrices for demo
    n_protein_features = 20
    n_molecular_features = 30
    
    X_train = pd.DataFrame(
        np.random.randn(len(df_train_final), n_protein_features + n_molecular_features),
        columns=[f'feature_{i}' for i in range(n_protein_features + n_molecular_features)]
    )
    X_val = pd.DataFrame(
        np.random.randn(len(df_val_final), n_protein_features + n_molecular_features),
        columns=[f'feature_{i}' for i in range(n_protein_features + n_molecular_features)]
    )
    X_test = pd.DataFrame(
        np.random.randn(len(df_test_final), n_protein_features + n_molecular_features),
        columns=[f'feature_{i}' for i in range(n_protein_features + n_molecular_features)]
    )
    
    print(f"Created demo feature matrices: {X_train.shape[1]} features")

print(f"\n🎯 Dataset ready for machine learning!")
print(f"📊 Summary:")
print(f"   • Total samples: {len(df_train_final) + len(df_val_final) + len(df_test_final)}")
print(f"   • Features: {X_train.shape[1] if 'X_train' in locals() else 'N/A'}")
print(f"   • Target: pKi/pKd/pIC50 (continuous)")
print(f"   • Model constraint: ≤50M parameters")
print(f"   • Approach: Pose-free binding affinity prediction")

print(f"\n📝 Next steps for hackathon:")
print(f"   1. Train lightweight ML models (Random Forest, XGBoost, small NN)")
print(f"   2. Implement protein language model embeddings (ESM-2)")
print(f"   3. Add molecular fingerprints and graph neural networks")
print(f"   4. Create ensemble models for robust predictions")
print(f"   5. Build interactive demo interface")